# Notes on Machine Learning Models - Part 1 - Feature Engineering

အပိုင်း ၃ ပိုင်းရှိတဲ့ ဒီ notes တွေမှာ အဓိကအားဖြင့် Machine Learning Model တွေကို အသုံးပြုရာမှာ သတိပြုရမဲ့ အောက်ပါ အကြောင်းအရာများကို ပြောပြပေးသွားမှာ ဖြစ်ပါတယ်။ 

* Feature Engineering 
* Model Engineering and
* Coding Best Practices

ဒီအပိုင်းက ပထမဆုံးအပိုင်း Feature Engineering ဖြစ်ပါတယ်။ 

$X$ ရော $y$ ပါရှိတဲ့ **supervised** Machine Learning ပဲ ဖြစ်ဖြစ်၊ $X$ တမျိုးပဲ ရှိတဲ့ **unsupervised** Machine Learning ပဲ ဖြစ်ဖြစ် Feature တွေ ရွေးချယ်မှုနဲ့ Engineering လုပ်တာတွေဟာ အရေးပါပါတယ်။ 

ဒါကြောင့် ဒီအပိုင်း မှာ 

* Feature Preprocessing Techniques
* Feature Selection Techniques တွေနဲ့ 
* Dimensionality Reduction/Increment Techniques တွေ အကြောင်း ပြောမယ်။

> Feature Engineering နဲ့ ဆိုင်ပေမဲ့ Data Type specific ဖြစ်တဲ့ Text/Image/Audio/Timeseries Data Manipulation ကိုတော့ Model Engineering နဲ့ အတူပြောမယ်။

## Feature Preprocessing

In [2]:
from sklearn import preprocessing as sk_pp
from sklearn import datasets as sk_ds

import numpy as np
import pandas as pd

In [3]:
df_X, ds_y = sk_ds.fetch_openml(name="credit-g", as_frame=True, return_X_y=True)
df_X.head()

/home/edward/anaconda3/envs/py37-dsup/lib/python3.7/site-packages/sklearn/datasets/_openml.py:421: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  " {version}.".format(name=name, version=res[0]["version"])


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


### Missing Values


ဒီ dataset မှာ null value `NaN` ပါ/မပါကို အရင် ကြည့်ရမယ်။ ကြည့်ပုံကြည့်နည်းကတော့ `pandas.DataFrame` ရဲ့ `isna` method နဲ့ပါပဲ။

In [ ]:
df_X.isna().any()

In [ ]:
ds_y.isna().any()

ဒီတော့ ဒီ data မှာ null value `NaN` မပါတာကို တွေ့ရမယ်။ ပါခဲ့ရင် `pandas.DataFrame.fillna` method နဲ့ ဖြည့်တာ (သို့မဟုတ်) `pandas.DataFrame.dropna` နဲ့ ဖြုတ်ချတာ တခုခု လုပ်ရမယ်။

### Feature Encoding


`sklearn` ပဲ ဖြစ်ဖြစ်၊ `XGBoost` ပဲ ဖြစ်ဖြစ်၊ `lightgbm` ပဲ ဖြစ်ဖြစ် ... အခု ခေတ်စားနေတဲ့ Deep Learning `pytorch`/`tensorflow` ပဲ ဖြစ်ဖြစ် numeric value မဟုတ်ရင် ဘာမှ လုပ်လို့ မရဘူး။ ဒါကြောင့် Numeric Value မဟုတ်တဲ့ Categorical Feature တွေကို Encode လုပ်ပေးရတယ်။

ဒီလို encode လုပ်ရာမှာ 

* One-Hot Encoding နဲ့ 
* Ordinal Encoding (Label Encoding) ဆိုပြီး ၂ မျိုး ရှိတယ်။

#### Knowing `dtypes`
ဘာပဲဖြစ်ဖြစ် encode မလုပ်ခင်မှာ ဘယ် column တွေက ဘာ data type လဲဆိုတာ သိဖို့လိုတယ်။ 

In [ ]:
df_X.dtypes

data type ပေါ်မူတည်ပြီး column တွေကို ရွေးနိုင်ဖို့ `pandas.DataFrame.select_dtypes` method ကို သုံးနိုင်တယ်။

In [4]:
df_X_category = df_X.select_dtypes(include=["category"])
df_X_category.head()

,checking_status,credit_history,purpose,savings_status,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker
0,<0,critical/other existing credit,radio/tv,no known savings,>=7,male single,none,real estate,none,own,skilled,yes,yes
1,0<=X<200,existing paid,radio/tv,<100,1<=X<4,female div/dep/mar,none,real estate,none,own,skilled,none,yes
2,no checking,critical/other existing credit,education,<100,4<=X<7,male single,none,real estate,none,own,unskilled resident,none,yes
3,<0,existing paid,furniture/equipment,<100,4<=X<7,male single,guarantor,life insurance,none,for free,skilled,none,yes
4,<0,delayed previously,new car,<100,1<=X<4,male single,none,no known property,none,for free,skilled,none,yes


In [5]:
df_X_number = df_X.select_dtypes(include=["number"])
df_X_number.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0
1,48.0,5951.0,2.0,2.0,22.0,1.0,1.0
2,12.0,2096.0,2.0,3.0,49.0,1.0,2.0
3,42.0,7882.0,2.0,4.0,45.0,1.0,2.0
4,24.0,4870.0,3.0,4.0,53.0,2.0,2.0


#### Ensuring data nature

Data nature ဆိုတာ categorical လား၊ numeric လားကို ဆိုလိုတယ်။ 

လက်တွေ့ဘဝမှာ computer ထဲ သိမ်းထားတဲ့ `dtypes` နဲ့ data nature က အမြဲတူမနေနိုင်ဘူး။

တချို့ dataset တွေမှာ numeric value ဖြစ်နေပေမဲ့ categorical data ဖြစ်နေတတ်တယ်။ 

> ဥပမာ McDonald's က အရောင်း transaction ဆိုရင် Combo 1, Combo 2, Combo 3 ကို 1, 2, 3 လို့ encode လုပ်ထားတဲ့ column ပါလာတာမျိုး။ ဒါကြောင့် some numbers may be categories လို့ အမြဲသတိထားရမယ်

ပြီးတော့ ... 

* Categorical မှာမှ order ရှိတဲ့ ordinal (XS/S/M/L/XL/XXL) လား၊ order မရှိတဲ့ norminal (blue/green/red) လား ထပ်ခွဲနိုင်ပြီး

* Numeric မှာမှ integer နဲ့ float ဆိုပြီး ထပ်ခွဲနိုင်တယ်။

ဒါတွေကို သိနိုင်ဖို့ `pandas.Series.unique` method နဲ့ `pandas.Series.nunique` method တွေကို သုံးပြီး ကြည့်နိုင်တယ်။

In [6]:
col_to_unique_values = {c: [df_X[c].unique()] for c in df_X.columns}
df_col_to_unique_values = pd.DataFrame(data=col_to_unique_values, index=["unique_values"]).T
df_col_to_unique_values.loc[:, "unique_count"] = df_col_to_unique_values.apply(lambda x : df_X[x.name].nunique(), axis=1)
df_col_to_unique_values

,unique_values,unique_count
checking_status,"['<0', '0<=X<200', 'no checking', '>=200'] Cat...",4
duration,"[6.0, 48.0, 12.0, 42.0, 24.0, 36.0, 30.0, 15.0...",33
credit_history,"['critical/other existing credit', 'existing p...",5
purpose,"['radio/tv', 'education', 'furniture/equipment...",10
credit_amount,"[1169.0, 5951.0, 2096.0, 7882.0, 4870.0, 9055....",921
savings_status,"['no known savings', '<100', '500<=X<1000', '>...",5
employment,"['>=7', '1<=X<4', '4<=X<7', 'unemployed', '<1'...",5
installment_commitment,"[4.0, 2.0, 3.0, 1.0]",4
personal_status,"['male single', 'female div/dep/mar', 'male di...",4
other_parties,"['none', 'guarantor', 'co applicant'] Categori...",3


အထူးသဖြင့် `unique_count` နည်းတဲ့ numeric column တွေကို အဓိကထားပြီး ကြည့်ရမယ်။ 

> ဒီ dataset မှာတော့ categorical nature နဲ့ numeric column မရှိဘူး။ ဥပမာ ... `residence_since` ဆိုရင် 4.0 က 2.0 ထက် ကြီးတယ်။ categorical မဟုတ်ဘူး။ 

တခါတလေမှာ `pandas.Series.unique` အစား `pandas.Series.value_counts` function ကို သုံးနိုင်တယ်။

In [ ]:
ds_y.value_counts()

#### Encoding


အလွယ်အားဖြင့် ordinal data တွေကို ordinal encode လုပ်ပြီး norminal data တွေကို one-hot encode လုပ်ရမယ်။

အခု ဒါတွေကို သိအောင် data ကို explore လုပ်ရမယ်။

In [ ]:
for c in df_X_category.columns:
    print ("Column : {}".format(c))
    print (df_X_category[c].dtype.categories)

ဒီနေရာမှာ မြင်သာတဲ့ ordinal data တွေက အောက်ပါတို့ ဖြစ်တယ်။ သူတို့နဲ့အတူ အစီအစဉ်ကိုပါ ပြထားတယ်။

* `credit_history` : `['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit']`
* `savings_status` : `['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']` -- ဒီနေရာမှာ ပြန်စီလိုက်တာကို သတိပြုပါ

တခြား ordinal data တွေ ရှိနေနိုင်သေးပေမဲ့ ကျန်တာကို ဒီနေ့အဖို့ one-hot encoding ပဲ လုပ်လိုက်မယ်။

> `personal_status` ကနေပြီး `gender` ထုတ်လို့ ရတာကို သတိထားပါ။ အိမ်စာအနေနဲ့ ထုတ်ကြည့်ပါ။

In [ ]:
df_features = pd.DataFrame(index=df_X.index, data=None)

ordinal_columns = ["credit_history", "savings_status"]
oe = sk_pp.OrdinalEncoder(
    # အောက်က categories parameter မှာ array-like of array-like (list of list) ထည့်ပေးရတာ သတိပြုပါ။
    categories=[
        ['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit'],
        ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']
    ], 
    # handle_unknown က သိပ်အရေးကြီးတယ်။ ဒါမပါသွားရင် production ကျမှ ပြဿနာ တက်တတ်တယ်။ default is "error"
    handle_unknown="use_encoded_value", 
    unknown_value=np.nan
)

df_features.loc[:, ["oe_{}".format(c) for c in ordinal_columns]] = oe.fit_transform(df_X[ordinal_columns])
df_features.head()

**Err** အပေါ်က cell က code မှာ သီအိုရီ/သဘောတရား အမှားတခု ပါနေတယ်။ 

> စာသင်ချိန်အတွင်းမှာပဲ ရအောင် ရှာပါ။

In [7]:
from sklearn import model_selection as sk_ms

df_X_tr, df_X_ts, ds_y_tr, ds_y_ts = sk_ms.train_test_split(df_X, ds_y, test_size=0.2, shuffle=True, random_state=42)

df_feat_tr = pd.DataFrame(data=None, index=df_X_tr.index)
df_feat_ts = pd.DataFrame(data=None, index=df_X_ts.index)

In [9]:
ordinal_columns = ["credit_history", "savings_status"]
oe = sk_pp.OrdinalEncoder(
    # အောက်က categories parameter မှာ array-like of array-like (list of list) ထည့်ပေးရတာ သတိပြုပါ။
    categories=[
        ['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit'],
        ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']
    ], 
    # handle_unknown က သိပ်အရေးကြီးတယ်။ ဒါမပါသွားရင် production ကျမှ ပြဿနာ တက်တတ်တယ်။ default is "error"
    handle_unknown="use_encoded_value", 
    unknown_value=np.nan
)
oe.fit(df_X_tr[ordinal_columns])
ordinal_features = ["oe_{}".format(c) for c in ordinal_columns]

df_feat_tr.loc[:, ordinal_features] = oe.transform(df_X_tr[ordinal_columns])
# see ? you can never ever fit with whole dataset; 
df_feat_ts.loc[:, ordinal_features] = oe.transform(df_X_ts[ordinal_columns])

df_feat_tr.head()

,oe_credit_history,oe_savings_status
29,3.0,1.0
535,4.0,1.0
695,2.0,3.0
557,0.0,0.0
836,2.0,0.0


In [10]:
norminal_columns = [c for c in df_X_category.columns if c not in ordinal_columns]

ohe = sk_pp.OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe.fit(df_X_tr[norminal_columns])

norminal_features = ohe.get_feature_names_out()
df_feat_tr.loc[:, norminal_features] = ohe.transform(df_X_tr[norminal_columns])
df_feat_ts.loc[:, norminal_features] = ohe.transform(df_X_ts[norminal_columns])
df_feat_tr.head()

,oe_credit_history,oe_savings_status,checking_status_0<=X<200,checking_status_<0,checking_status_>=200,checking_status_no checking,purpose_business,purpose_domestic appliance,purpose_education,purpose_furniture/equipment,...,housing_own,housing_rent,job_high qualif/self emp/mgmt,job_skilled,job_unemp/unskilled non res,job_unskilled resident,own_telephone_none,own_telephone_yes,foreign_worker_no,foreign_worker_yes
29,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
535,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
695,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
557,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
836,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


### Scaling

Encoding ပြီးသွားတော့ Scaling ပေါ့။ 

Scaling မှာ `StandardScaler`, `MinMaxScaler` နဲ့ `MaxAbsScaler` ဆိုပြီး ရှိတယ်။ 

* `StandardScaler` က Standardization (or) Mean Removal လုပ်ဖို့ သုံးတယ်။ mean ကို နှုတ်ပြီး standard deviation နဲ့ စားတာပါပဲ။
* `MinMaxScaler` ကျတော့ အငယ်ဆုံးကို 0 (သုည)၊ အကြီးဆုံးကို 1 (တစ်) အတွင်း ဝင်အောင် scale လုပ်တာ။ min ကို နှုတ်ပြီး (max - min) နဲ့ စားတာပါပဲ။
* `MaxAbsScaler` ကျတော့ အကြီးဆုံးကို 1 (တစ်) ဖြစ်အောင် scaler လုပ်တာ။ တန်ဖိုးကို max နဲ့ စားတာပါပဲ။

ဒီနေရာမှာ scale လုပ်တာ မဟုတ်ပဲ Data တွေကို နေရာရွှေ့ပေးတဲ့ နောက်ထပ် transformer တမျိုးကိုလဲ မှတ်ထားသင့်တယ်။ သူက `QuantileTransformer`။ သူ့မှာ variant ၂ မျိုးရှိတယ်။ `output_distribution="uniform"` နဲ့ `output_distribution="normal"` ဆိုပြီးတော့။

In [11]:
ss = sk_pp.StandardScaler()
mms = sk_pp.MinMaxScaler()
mas = sk_pp.MaxAbsScaler()

numerical_columns = list(df_X_number.columns) + ordinal_features
ss_features = ["ss_{}".format(c) for c in numerical_columns]
mms_features = ["mms_{}".format(c) for c in numerical_columns]
mas_features = ["mas_{}".format(c) for c in numerical_columns]

In [12]:
df_X_tr.loc[:, ordinal_features] = df_feat_tr[ordinal_features]
df_X_ts.loc[:, ordinal_features] = df_feat_ts[ordinal_features]

df_feat_tr.drop(ordinal_features, axis="columns", inplace=True)
df_feat_ts.drop(ordinal_features, axis="columns", inplace=True)

In [13]:
df_feat_tr.loc[:, ss_features] = ss.fit_transform(df_X_tr[numerical_columns])
df_feat_tr.loc[:, mms_features] = mms.fit_transform(df_X_tr[numerical_columns])
df_feat_tr.loc[:, mas_features] = mas.fit_transform(df_X_tr[numerical_columns])

df_feat_ts.loc[:, ss_features] = ss.fit_transform(df_X_ts[numerical_columns])
df_feat_ts.loc[:, mms_features] = mms.fit_transform(df_X_ts[numerical_columns])
df_feat_ts.loc[:, mas_features] = mas.fit_transform(df_X_ts[numerical_columns])

df_feat_tr.head()

,checking_status_0<=X<200,checking_status_<0,checking_status_>=200,checking_status_no checking,purpose_business,purpose_domestic appliance,purpose_education,purpose_furniture/equipment,purpose_new car,purpose_other,...,mms_oe_savings_status,mas_duration,mas_credit_amount,mas_installment_commitment,mas_residence_since,mas_age,mas_existing_credits,mas_num_dependents,mas_oe_credit_history,mas_oe_savings_status
29,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.25,1.00,0.371038,0.75,1.00,0.840000,0.50,0.5,0.75,0.25
535,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.25,0.35,0.125868,0.50,0.25,0.440000,0.25,0.5,1.00,0.25
695,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.75,0.10,0.067086,0.50,1.00,0.666667,0.25,0.5,0.50,0.75
557,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.00,0.35,0.271548,0.25,1.00,0.386667,0.50,0.5,0.00,0.00
836,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.20,0.048089,1.00,0.50,0.280000,0.25,0.5,0.50,0.00


တချို့ model တွေက scaling လုပ်ထားတဲ့ feature နဲ့မှ အဆင်ပြေပြီး တချို့ model တွေက မလိုဘူး။

> လိုတဲ့ model တွေက linear model တွေနဲ့ svm/svc model တွေဖြစ်တယ်။
>
> ဘယ် model တွေက scaling လုပ်ပေးစရာ မလိုဘူးလဲ ? 

နောက်ဆုံးအနေနဲ့ label ကို encode လုပ်ရမယ်။

In [14]:
le = sk_pp.LabelEncoder()
y_tr = le.fit_transform(ds_y_tr)
y_ts = le.transform(ds_y_ts)
le.classes_

array(['bad', 'good'], dtype=object)

ကျနော်တို့တွေက `"bad"` ကို ရှာချင်တာ ဖြစ်တာမို့ label ကို 1 to 0 and 0 to 1 လှည့်ပါမယ်။

In [15]:
y_tr = 1 - y_tr
y_ts = 1 - y_ts

## Feature Selection

Feature Selection လို့ ပြောသော်လည်းပဲ လိုအပ်ရင် preprocessing ကို ပြန်သွားရမှာ ဖြစ်တယ်။ 

ဒီ Section မှာလဲ ရှေ့ကို ကျော်ပြီး baseline သဘောနဲ့ modelling လုပ်တာ ရှိတယ်။

> ဒါတွေကြောင့်မို့လို့ Data Science Process ဟာ iterative/recursive process ဖြစ်တယ်။ တခုပြီးမှ နောက်တခု သွားတာမျိုး မရှိဘူး။

### Univariate Feature Selection

အရိုးဆုံး feature selection technique က univariate feature selection ဖြစ်တယ်။ အဲဒီအတွက် `SelectKBest` သို့မဟုတ် `SelectPercentile` ကို သုံးတယ်။

In [16]:
from sklearn import feature_selection as sk_fs

skb = sk_fs.SelectKBest(score_func=sk_fs.f_classif, k=50)
skb.fit(df_feat_tr, y_tr)
skb.get_feature_names_out()

array(['checking_status_0<=X<200', 'checking_status_<0',
       'checking_status_>=200', 'checking_status_no checking',
       'purpose_business', 'purpose_education', 'purpose_new car',
       'purpose_other', 'purpose_radio/tv', 'purpose_used car',
       'employment_4<=X<7', 'employment_<1', 'employment_>=7',
       'employment_unemployed', 'personal_status_female div/dep/mar',
       'personal_status_male div/sep', 'personal_status_male single',
       'other_parties_guarantor', 'property_magnitude_no known property',
       'property_magnitude_real estate', 'other_payment_plans_bank',
       'other_payment_plans_none', 'housing_for free', 'housing_own',
       'housing_rent', 'job_high qualif/self emp/mgmt',
       'job_unskilled resident', 'foreign_worker_no',
       'foreign_worker_yes', 'ss_duration', 'ss_credit_amount',
       'ss_installment_commitment', 'ss_age', 'ss_existing_credits',
       'ss_oe_credit_history', 'ss_oe_savings_status', 'mms_duration',
       'mms_credit_

### Model-based Feature Selection

ကိုယ်သုံးမဲ့ model ကို ကြိုသိရင် model နဲ့ Feature Selection လုပ်၊ ပြီးရင် Model မှာ ပြန်စမ်း ... ရလာတာနဲ့ Feature Selection ပြန်လုပ်လို့ရတယ်။

In [17]:
from sklearn import linear_model as sk_lm 
sfm = sk_fs.SelectFromModel(estimator=sk_lm.SGDClassifier(penalty="elasticnet", max_iter=10000, n_jobs=4), threshold="median", prefit=False)
sfm.fit(df_feat_tr, y_tr)
df_feat_tr.columns[sfm.get_support()]

Index(['checking_status_0<=X<200', 'checking_status_<0',
       'checking_status_>=200', 'checking_status_no checking',
       'purpose_domestic appliance', 'purpose_education', 'purpose_new car',
       'purpose_repairs', 'purpose_retraining', 'purpose_used car',
       'employment_1<=X<4', 'employment_>=7', 'personal_status_male div/sep',
       'personal_status_male single', 'other_parties_co applicant',
       'other_parties_guarantor', 'property_magnitude_car',
       'property_magnitude_no known property',
       'property_magnitude_real estate', 'other_payment_plans_bank',
       'other_payment_plans_none', 'other_payment_plans_stores',
       'housing_rent', 'job_high qualif/self emp/mgmt', 'job_skilled',
       'job_unemp/unskilled non res', 'job_unskilled resident',
       'foreign_worker_no', 'foreign_worker_yes', 'ss_credit_amount',
       'ss_installment_commitment', 'ss_age', 'ss_existing_credits',
       'ss_oe_credit_history', 'mms_installment_commitment',
       'mas_i

သတိထားရမှာက `SelectFromModel` က model တိုင်းနဲ့ အလုပ်မလုပ်ဘူး။ ဥပမာ SVM model တွေနဲ့ အလုပ်မလုပ်ဘူး။

> ဘာဖြစ်လို့လဲဆိုတော့ `coef_` or `feature_importances_` attributes တွေ model မှာ ရှိမှ အလုပ်လုပ်တယ်။

### Sequential Feature Selection

ဒီတခုကတော့ model တိုင်းနဲ့ အလုပ်လုပ်တယ်။

> သို့သော် ... ပိုကြာတယ်။

In [18]:
from sklearn import svm

nusvc = svm.NuSVC(class_weight={0: 1, 1: 5}, random_state=42)
sfs = sk_fs.SequentialFeatureSelector(
    estimator=nusvc, 
    n_features_to_select=50, 
    # backward means start with all and remove
    # forward means start with 1 and add
    direction="backward", 
    cv=10, 
    n_jobs=4
)
sfs.fit(df_feat_tr, y_tr)
df_feat_tr.columns[sfs.get_support()]

Index(['checking_status_0<=X<200', 'checking_status_<0',
       'checking_status_>=200', 'checking_status_no checking',
       'purpose_business', 'purpose_furniture/equipment', 'purpose_new car',
       'purpose_radio/tv', 'purpose_used car', 'employment_1<=X<4',
       'employment_4<=X<7', 'employment_<1', 'employment_>=7',
       'employment_unemployed', 'personal_status_female div/dep/mar',
       'personal_status_male div/sep', 'personal_status_male mar/wid',
       'personal_status_male single', 'other_parties_co applicant',
       'other_parties_guarantor', 'other_parties_none',
       'property_magnitude_car', 'property_magnitude_life insurance',
       'property_magnitude_no known property',
       'property_magnitude_real estate', 'other_payment_plans_bank',
       'other_payment_plans_none', 'housing_for free', 'housing_own',
       'housing_rent', 'job_high qualif/self emp/mgmt', 'job_skilled',
       'job_unskilled resident', 'own_telephone_none', 'own_telephone_yes',
    

## Dimensionality Reduction/Increment

ဒီနောက်ဆုံးပိုင်းကတော့ dimensionality (number of column) ကို လျှော့/တိုးတဲ့ အပိုင်းပါပဲ။ 

Dimensionality Reduction အတွက်

* KMeans
* SVD နဲ့
* PCA တို့ကို သုံးလို့ရပြီး ... 

Dimensionality Increment အတွက်ကတော့ KMeans တမျိုးတည်းကို သုံးလို့ ရပါတယ်။

> Time-series တွေကို dimensionality reduction လုပ်ဖို့ သုံးတာက ဘာပါလိမ့်နော် ... ?

In [19]:
from sklearn import decomposition as sk_decom

svd = sk_decom.TruncatedSVD(n_components=20, n_iter=10, random_state=42)
pca = sk_decom.PCA(n_components=20, random_state=42)

svd.fit(df_feat_tr.values)
pca.fit(df_feat_tr.values)

PCA(n_components=20, random_state=42)

အခုနောက်ပိုင်း အသစ်ပေါ်တာအနေနဲ့ t-SNE ဟာ SVD/PCA တို့လို linear method မဟုတ်ပဲ non-linear method ဖြစ်တယ်။ 

> သူအလုပ်လုပ်ပုံက SVM နဲ့ ပြောင်းပြန်လို့ အလွယ်မှတ်နိုင်တယ်။ 

> t-SNE ထက် မြန်တဲ့ UMAP ဆိုတာလဲ ရှိသေးတယ်။

In [22]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, random_state=42, n_jobs=4)
tsne = tsne.fit(df_feat_tr.values)
tsne.kl_divergence_

/home/edward/anaconda3/envs/py37-dsup/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/edward/anaconda3/envs/py37-dsup/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


1.5189365148544312

kl_divergence_ (cross entropy ပဲ ဆိုကြပါစို့) က နည်းလေ information များလေ၊ similarity betweenလို့ ယူဆနိုင်တယ်။

> သတိထားရမှာက t-SNE ဟာ visualization ကို support လုပ်ဖို့ ထုတ်ထားတဲ့ algorithm မို့လို့ n_component က 3 ထက် ကြီးလို့ မရဘူး။

> လေ့ကျင့်ခန်းအနေနဲ့ အပေါ်က ရထားတဲ့ feature ၅၀ (ကြိုက်ရာ ၅၀) နဲ့ svd/pca တခုက feature 20 တို့ ပေါင်းပြီး model တခု ဆောက်ကြည့်ပါ။

### KMeans

In [23]:
from sklearn import cluster as sk_clus

kmeans = sk_clus.KMeans(n_clusters=20, max_iter=1000, random_state=42)

kmeans.fit(df_feat_tr.values)

KMeans(max_iter=1000, n_clusters=20, random_state=42)

# Misc. Proofs

### Why Euclidean Distance of Normalized Vectors $\propto$ Cosine Distance

Let us start with definitions.

Cosine can be derived from definition of dot-product:

$X \cdot Y = cos(\theta)\|X\|\|Y\|$

$\Rightarrow cos(\theta) = \frac{X \cdot Y}{\|X\|\|Y\|}$ 

> Also take note that dot-product $X \cdot Y$ is a scalar and is the same as $X^T Y$

When normalized, both $\|X\|$ and $\|Y\|$ becomes 1 $\because$ it is the definition of normalization.

$\therefore cos(\theta) = X \cdot Y$ when both $X$ and $Y$ are normalized unit vectors.

> The above $cos(\theta)$ is cosine similarity because $cos(0)=1$ and $cos(\pi/2)=0$.
>
> Cosine distance is given by $1 - cos(\theta)$.

Euclidean distance is Norm-2 of $X-Y$. 

$dist(X, Y) = \|X - Y\|_2$

Let's square it.

$dist(X, Y)^2 = \|X - Y\|_2^2$

$= (X-Y)^T(X-Y)$

> $ \because V^T V = \sum{v_i^2}$ by definition.

$= (X^T-Y^T)(X-Y)$

$= (X^T X) - (X^T Y) - (Y^T X) + (Y^T Y)$ 

> note that the two terms in middle are scaler and the same, i.e. $(X^T Y) = (Y^T X)$.

$= (1) - (X^T Y) - (X^T Y) + (1)$

$= 2 - 2 (X^T Y)$

$= 2 (1 - X^T Y)$

$= 2 (1 - X \cdot Y)$

$= 2 (1 - cos(\theta))$

$= 2 \times$ cosine distance.

In [ ]:
df_feat_ts.head()